In [1]:
%load_ext autoreload
%autoreload 2
# %load_ext line_profiler
import h5py
import randomcolor
import numpy as np

from utils import *
from optimization import *
from topological_graph import *

from scipy.spatial import cKDTree
from scipy.interpolate import UnivariateSpline, splev, splprep
from scipy.optimize import minimize
from glob import glob
from tqdm import tqdm
import point_cloud_utils as pcu
import torch
from torch_geometric.nn import fps as tfps
import time
from joblib import Parallel, delayed
from sklearn.model_selection import ParameterGrid 
from sklearn.neighbors import KNeighborsRegressor
from pytorch3d.loss.chamfer import chamfer_distance
from geomloss import SamplesLoss
import copy
from sklearn.metrics import mean_squared_error
import os
import pandas as pd

In [3]:
# parameters

RES = 0.02
sharpness_threshold = RES * 3 #RES*3

knn_radius = 2

filtering_radius = RES * 1 # max distance to connect a pair in knn filtering
corner_connected_components_radius = RES * knn_radius # max distance to connect a pair in knn corner separation
curve_connected_components_radius = RES * 2.5 # max distance to connect a pair in knn curve separation

sampling = 'blue' # 'blue','random','voxel'
num_voxels_per_axis = 100
subsample_factor = 10
filtering_factor = 30
filtering_mode = False
fps_factor = 5

# corner_detector_radius = RES * 8
# upper_variance_threshold = 0.05
# lower_variance_threshold = 0.04
cornerness_threshold = 0.7
# box_margin = RES * 4
quantile = 1

curve_extraction_mode = 'knn' # 'subdivision', 'knn'
endpoint_detector_radius = RES * 10
endpoint_threshold = 0.6

corner_connector_radius = RES * 20

initial_split_threshold = RES * 4
# optimization_split_threshold = RES * 3
alpha_ang = 1000

DISPLAY_RES = 0.03
cut_metric = 'fscore' #fscore #'chamfer_dist','sinkhorn' 


small_lines_clip_primitive_number = 3
small_lines_clip_primitives_length = 0.4

id_it = 0

In [4]:
path = 'input/'
_ids = np.sort(glob('input/*.hdf5'))
# _id = 'abc_0051_00514480_6c33de245ad4c4ff41a3360f_000'
_id = _ids[id_it].split('__')[0][6:]
_id

'abc_0050_00500041_5aa40dcd43fa0b14df9bdcf8_010'

In [4]:
# path = 'input_new_models/'
# _ids = np.sort(glob('input_new_models/*.hdf5'))
# # _id = 'abc_0051_00514480_6c33de245ad4c4ff41a3360f_000'
# # _id = _ids[id_it].split('__')[0][6:]
# _id = _ids[id_it].split('__')[0][17:]
# _id

'abc_0051_00517802_486ace821ef499b593f72d64_000'

In [5]:
meta_exp_name = 'cor_' + str(cornerness_threshold) + '_sharp_' + str(sharpness_threshold)+'_subs_'+ str(subsample_factor)

In [6]:
import os
if not os.path.exists(meta_exp_name):
    os.makedirs(meta_exp_name)

In [7]:
import os
#changed
folder_name = 'id_it' + str(id_it) + '_'+ str(_id)+'_cor_' + str(cornerness_threshold) + '_sharp_' + str(sharpness_threshold)+'_subs_'+ str(subsample_factor)
if not os.path.exists(meta_exp_name+'/'+folder_name):
    os.makedirs(meta_exp_name+ '/'+folder_name)

In [8]:
### print('Processing ', _id)
# with h5py.File('{path}/{_id}__min.hdf5'.format(path=path, _id=_id), 'r') as f:
with h5py.File('{path}/{_id}__adv60.hdf5'.format(path=path, _id=_id), 'r') as f:
    whole_model_points = f['points'][:]
    whole_model_distances = f['distances'][:]
points = whole_model_points[whole_model_distances < sharpness_threshold]
distances = whole_model_distances[whole_model_distances < sharpness_threshold]
print(points.shape)
subsample_rate = int(points.shape[0] / subsample_factor)
print("subsample_rate", subsample_rate)

print('processing {size} points'.format(size=len(points)))
start_all = time.time()
if sampling:
    if  subsample_rate > 0 and sampling=='random':
        print('random sampling')
        sub_idx = np.random.choice(np.arange(len(points)), subsample_rate)
        points = points[sub_idx]
        distances = distances[sub_idx]
    elif sampling=='blue'and subsample_rate > 0:
        start =time.time()
        print('blue sampling')
        # Downsample a point cloud by approximately to subsample_rate so that the sampled points approximately
        # follow a blue noise distribution
        # idx is an array of integer indices into v indicating which samples to keep
        sub_idx = pcu.downsample_point_cloud_poisson_disk(points, num_samples=subsample_rate)
        # Use the indices to get the sample positions and normals
        points = points[sub_idx]
        distances = distances[sub_idx]
        print(time.time()-start)
    elif sampling=='voxel':
        start =time.time()
        print("voxel sampling")
        # We'll use a voxel grid with 128 voxels per axis
        distances = np.array(np.tile(distances, (3,1)).transpose(1,0), order='C')

        # Size of the axis aligned bounding box of the point cloud
        bbox_size = points.max(0) - points.min(0)

        # The size per-axis of a single voxel
        sizeof_voxel = bbox_size / num_voxels_per_axis

        # Downsample a point cloud on a voxel grid so there is at most one point per voxel.
        # Multiple points, normals, and colors within a voxel cell are averaged together.
        v_sampled, n_sampled, c_sampled = pcu.downsample_point_cloud_voxel_grid(sizeof_voxel, points,distances , None)
        points = v_sampled
        distances = n_sampled[:,0]
        print(time.time()-start)
    else:
        raise Exception("No other option for samoling") 


if filtering_mode:
    print('filtering')
    filtered_clusters = separate_graph_connected_components(points, radius=filtering_radius, filtering_mode=True, 
                                                            filtering_factor=filtering_factor)
    points = points[np.unique(np.concatenate(filtered_clusters))]
    distances = distances[np.unique(np.concatenate(filtered_clusters))]

print('processing {size} points'.format(size=len(points)))
# fps = farthest_point_sampling(points, points.shape[0] // fps_factor)
# print('fps_finished')
print(time.time()-start_all)

(182904, 3)
subsample_rate 18290
processing 182904 points
blue sampling
2.379896879196167
processing 17925 points
2.380798578262329


In [9]:
start = time.time()
a = torch.tensor(points)
print(time.time()-start)
start = time.time()
fps = tfps(a,ratio=1/fps_factor)
print(time.time()-start)
fps = fps.numpy()

0.13739466667175293
1.2987494468688965


# New methods

In [10]:
parameters = {'corner_detector_radius':[RES * 5, RES * 6, RES * 7, RES * 8],
              'neighbours_count':[10,20,40], 
              'variance_dynamics_threshold':[5,10,15,20,25]}

In [11]:
def neighb_prob(par, variance):
    var_dynamics = np.zeros_like(variances)
    tree_fps = cKDTree(points[fps])
    fps_nbhds = tree_fps.query(points[fps], par['neighbours_count'])
    
    for i in range(1,par['neighbours_count']):
        var_dynamics += ((variances[fps_nbhds[1][:,i]] - variances) / fps_nbhds[0][:,i][:,None]) ** 2
#     var_dynamics = np.abs(var_dynamics)
    # it can be done softer, can be just averaging var_dinamics
    return np.where(var_dynamics.sum(-1) > par['variance_dynamics_threshold'])
    #return var_dynamics

In [12]:
boxes = []
for it in range(len(parameters['corner_detector_radius'])):
    tree = cKDTree(points)
    neighbours = tree.query_ball_point(points[fps], r=parameters["corner_detector_radius"][it])
    variances = []
    for n in tqdm(neighbours):
        pca = PCA(3)
        # if size of neighbourhood is sufficient
        if len(n) > 5:
            pca.fit(points[n])
            variances.append(pca.explained_variance_ratio_)
        else: variances.append([1,0,0])
    variances = np.array(variances)
    keys = ['neighbours_count','variance_dynamics_threshold' ]
    cicle_par = {x:parameters[x] for x in keys}
    #TODO can be optimized 
    boxsss=Parallel(n_jobs=4)(delayed(neighb_prob)(par,variances) for par in tqdm(ParameterGrid(cicle_par)))
    boxes.append(boxsss)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 1201.23it/s]


In [13]:
(corners,counts) = np.unique(np.concatenate(np.concatenate(np.concatenate(boxes))),return_counts=True)
prob = counts/(len(list(ParameterGrid(cicle_par)))*len(parameters['corner_detector_radius']))

<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [14]:
plot = k3d.plot()

cmap=k3d.colormaps.matplotlib_color_maps.plasma
colors = k3d.helpers.map_colors(
                prob, cmap, [prob.min(), prob.max()]
            ).astype(np.uint32)

k3d_points = k3d.points(points[fps][corners.astype(int)], point_size=DISPLAY_RES, shader='3d', name='sharp_points',colors=colors)
plot += k3d_points

k3d_points_i = k3d.points(points[fps][~np.in1d(range(len(points[fps])),corners)], point_size=DISPLAY_RES, shader='3d', name='sharp_points')
plot += k3d_points_i

plot.display()

with open(meta_exp_name+'/'+folder_name+'/'+'prob_corr'+'.html', 'w') as f:
    f.write(plot.get_snapshot())

/trinity/home/v.egiazarian/environments/envs/lib/python3.8/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


Output()

In [15]:
all_fps_prob = np.zeros(points[fps].shape[0])
all_fps_prob[corners] = prob

In [16]:
neigh = KNeighborsRegressor(n_neighbors=50)

In [17]:
neigh.fit(points[fps], all_fps_prob)

KNeighborsRegressor(n_neighbors=50)

In [18]:
# не знаю стоит ли предсказывать для всех или только для тех которые не известны
all_prob = neigh.predict(points)

In [19]:
# cornerness_threshold=0.75

In [20]:
# cornerness_threshold=0.7

In [21]:
corners = np.arange(points.shape[0])[all_prob>cornerness_threshold]

In [26]:
cornerness_threshold=0.6

In [27]:
plot = k3d.plot()


k3d_points = k3d.points(points[np.setdiff1d(np.arange(len(points)), corners)], point_size=DISPLAY_RES, shader='3d', name='sharp_points')
plot += k3d_points

k3d_points = k3d.points(points[corners], point_size=DISPLAY_RES, shader='3d', name='sharp_points',color=0xff0000)
plot += k3d_points


plot.display()
with open(meta_exp_name + '/'+ folder_name +'/'+'corners'+'.html', 'w') as f:
    f.write(plot.get_snapshot())

Output()

In [28]:
# decrease if neccessary (if there are glued corners)
# quantile = 1

In [29]:
corner_centers = []
corner_clusters = []
corners = np.arange(points.shape[0])[all_prob>cornerness_threshold]

tmp_corner_clusters = separate_graph_connected_components(points[corners], corner_connected_components_radius,
                                                                          compute_barycenters=False)
tmp_corner_centers = [np.median(points[corners][clust], axis=0).tolist() for clust in tmp_corner_clusters] 

# for every corner box, sample two points as corners, and store connections between them

init_connections = []
norms = []
for i, cluster in enumerate(tmp_corner_clusters):
    tmp_p = points[corners][cluster]
    tmp_p -= tmp_p.mean(0)
    norms.append(np.linalg.norm(tmp_p, axis=1).max())
for i, cluster in enumerate(tmp_corner_clusters):
    if norms[i] < np.quantile(norms, quantile):
        corner_clusters.append(cluster)
        query = cKDTree(points[corners]).query(tmp_corner_centers[i], 1)
        corner_centers.append(query[1])
        continue
    else:
        gmm = GaussianMixture(2)
        res = gmm.fit_predict(points[corners][cluster])
        corner_clusters.append(np.array(cluster)[res == 0].tolist())
        corner_clusters.append(np.array(cluster)[res == 1].tolist())
        endpoints_query = cKDTree(points[corners]).query(gmm.means_, 1)
        ind = len(corner_centers)
        corner_centers.append(endpoints_query[1][0])
        corner_centers.append(endpoints_query[1][1])
        init_connections.append([ind, ind+1])    



8it [00:00, 29.10it/s]


In [30]:
not_corners = np.setdiff1d(np.arange(len(points)), corners)

# print('separating curves')
curves = separate_graph_connected_components(points[not_corners], radius=curve_connected_components_radius)

13it [00:02,  6.20it/s]


In [31]:
plot = k3d.plot()
rand_color = randomcolor.RandomColor()

cmap=k3d.colormaps.matplotlib_color_maps.plasma_r
colors = k3d.helpers.map_colors(
                distances, cmap, [distances.min(), distances.max()]
            ).astype(np.uint32)

plot += k3d.points(points, point_size=0.03, colors=colors)

for i in corner_clusters:
    color = rand_color.generate()[0]
    color = int('0x' + color[1:], 16)
    plot += k3d.points(points[corners][i], color=color, point_size=0.03)
    
for i in range(len(curves)):
    color = rand_color.generate()[0]
    color = int('0x' + color[1:], 16)
    plot += k3d.points(points[not_corners][curves[i]], color=color, point_size=0.03)
    
plot.display()
with open(meta_exp_name+'/'+folder_name+'/'+'segmentation_'+curve_extraction_mode+'.html', 'w') as f:
    f.write(plot.get_snapshot())

/trinity/home/v.egiazarian/environments/envs/lib/python3.8/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


Output()

In [28]:
# curve_extraction_mode = 'knn' #subdivision,knn

In [32]:
print('initializing topological graph')
from topological_graph import *
corner_positions, corner_pairs = initialize_topological_graph(points, distances, 
                                                              not_corners, curves, 
                                                              corners, corner_centers,
                                                              init_connections,
                                                              endpoint_detector_radius, endpoint_threshold, 
                                                              initial_split_threshold, corner_connector_radius,
                                                              curve_extraction_mode)

initializing topological graph


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:03<00:00,  4.11it/s]


In [33]:
# np.arange(len(corner_positions))
not_corners_ss = np.setdiff1d(np.arange(len(np.arange(len(corner_positions)))), np.unique(corner_pairs))

In [34]:
plot = k3d.plot()

k3d_points = k3d.points(points, point_size=DISPLAY_RES, opacity=0.1, shader='3d', name='sharp_points')
plot += k3d_points

points_corner_centers = k3d.points(np.array(corner_positions),
                                   color=0xFF0000, point_size=DISPLAY_RES, shader='3d', name='polyline_nodes')
plot += points_corner_centers

for edge in corner_pairs:
    try:
        e = k3d.line(np.array(corner_positions)[edge], name='polyline_edge')
        plot+=e
    except:
        continue
    
plot.display()


with open(meta_exp_name+'/'+folder_name+'/'+'graph_initialization_'+curve_extraction_mode+'.html', 'w') as f:
    f.write(plot.get_snapshot())

/trinity/home/v.egiazarian/environments/envs/lib/python3.8/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


Output()

In [35]:
print('optimizing topological graph')
from optimization import *
corner_positions, corner_pairs = optimize_topological_graph(corner_positions, corner_pairs, corner_centers, 
                                                            points, distances, 
                                                            initial_split_threshold, alpha_fit=1, 
                                                            alpha_ang=1, corner_alpha_ang=0)

optimizing topological graph


Optimization: step #149, fit loss: 6.6, angle loss: -18.9: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 150/150 [00:06<00:00, 23.71it/s]
Optimization: step #149, fit loss: 6.0, angle loss: -18.8: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 150/150 [00:06<00:00, 24.79it/s]


In [36]:
plot = k3d.plot()

k3d_points = k3d.points(points, point_size=DISPLAY_RES, opacity=0.1, shader='3d', name='sharp_points')
plot += k3d_points

points_corner_centers = k3d.points(np.array(corner_positions),
                                   color=0xFF0000, point_size=DISPLAY_RES, shader='3d', name='polyline_nodes')
plot += points_corner_centers

for edge in corner_pairs:
    try:
        e = k3d.line(np.array(corner_positions)[edge], name='polyline_edge')
        plot+=e
    except:
        continue
    
plot.display()


with open(meta_exp_name+'/'+folder_name+'/'+'graph_optimization_'+curve_extraction_mode+'.html', 'w') as f:
    f.write(plot.get_snapshot())

Output()

In [37]:
from optimization import *
print('creating paths')
_, paths, _, closed_paths, _ = get_paths_and_corners(corner_pairs, corner_positions, corner_centers)


print('aabboxing')
# aabboxes = create_aabboxes(np.array(corner_positions)[np.array(corner_pairs)])
matching = parallel_nearest_point(np.array(corner_positions), np.array(corner_pairs), points)
curve_data, curve_distances = recalculate(points, distances, matching, corner_pairs)

creating paths
aabboxing


In [38]:
# curve_data

In [39]:
# utils for straight segments detection

straight_threshold = 0.977
breakpoint_threshold = 0.2

path_angles = []
for path in paths:
    path_pairs = list(zip(path, path[1:]))
    current_angles = []
    if len(path_pairs) == 1:
        path_angles.append(None)
        continue
    for path_node in range(1,len(path) - 1):
        vec1 = np.array(corner_positions)[path[path_node]] - np.array(corner_positions)[path[0]]
        vec2 = np.array(corner_positions)[path[-1]] - np.array(corner_positions)[path[path_node]]
        vec1 /= np.linalg.norm(vec1)
        vec2 /= np.linalg.norm(vec2)
        current_angles.append(np.dot(vec1, vec2))
    path_angles.append(current_angles)
    
straight = np.zeros((len(paths)))
for i in range(len(paths)):
    if len(paths[i]) == 2:
        straight[i] = 1
    if path_angles[i] is not None:
        if (np.array(path_angles[i]) >= straight_threshold).all():
            straight[i] = 1

path_angles = []
for path in paths:
    path_pairs = list(zip(path, path[1:]))
    current_angles = []
    if len(path_pairs) == 1:
        path_angles.append(None)
        continue
    for path_node in range(1,len(path) - 1):
        vec1 = np.array(corner_positions)[path[path_node]] - np.array(corner_positions)[path[path_node-1]]
        vec2 = np.array(corner_positions)[path[path_node+1]] - np.array(corner_positions)[path[path_node]]
        vec1 /= np.linalg.norm(vec1)
        vec2 /= np.linalg.norm(vec2)
        current_angles.append(np.dot(vec1, vec2))
    path_angles.append(current_angles)

In [40]:
        
new_paths = []
for i,path in enumerate(paths):
    if path_angles[i] is None:
        new_paths.append(path)
        continue
    if straight[i] == 1:
        new_paths.append(path)
        continue
    if (np.array(path_angles[i]) > straight_threshold).any() and (np.array(path_angles[i]) < breakpoint_threshold).any():
        breakpoints = []
        for j in range(1,len(path)-2):
            if (np.array(path_angles[i]) > straight_threshold)[j-1] == (np.array(path_angles[i]) < breakpoint_threshold)[j]:
                breakpoints.append((np.array([j-1,j]) * np.logical_not([(np.array(path_angles[i]) > straight_threshold)[j-1], (np.array(path_angles[i]) > breakpoint_threshold)[j]])).sum())
        if not breakpoints:
            continue
        breakpoints = np.unique(breakpoints)
        new_paths.append(path[:breakpoints[0]+2])
        new_paths.append(path[breakpoints[-1]+1:])
        if len(breakpoints) > 1:
            for b,c in zip(breakpoints[:-1], breakpoints[1:]):
                new_paths.append(path[b+1:c+2])
    else:
        new_paths.append(path)

In [41]:
paths = new_paths

In [42]:
# def process_paths(paths, corner_pairs, corner_positions, curve_data, curve_distances):
new_paths = []
path_points = []
path_distances = []
path_params = []
path_knots = []
avg_len = np.linalg.norm(np.array(corner_positions)[
    np.array(corner_pairs)[:,0]] - np.array(corner_positions)[
    np.array(corner_pairs)[:,1]], axis=1).mean()
for path in paths:
    path_len = len(path)
#     print(len(path))
    path_pairs = list(zip(path, path[1:]))
    current_points = []
    current_distances = []
    current_params = []
    current_knots = []
    knot_param = 0
    for i,current_pair in enumerate(path_pairs):
        endpoints = np.array(corner_positions)[np.array(current_pair)]
        ind = np.where(np.isin(corner_pairs, current_pair).sum(1) == 2)[0][0]
        if len(path) < 3 and len(curve_data[ind]) < 5:
            break
        cosines = np.dot(curve_data[ind] - endpoints[0], endpoints[1] - endpoints[0]) / np.linalg.norm(endpoints[1] - endpoints[0])
        truncate = np.logical_and(cosines >= 0, cosines <= np.linalg.norm(endpoints[1] - endpoints[0]))

        param = cosines[truncate]

        argsort = np.argsort(param)
        if len(current_params) == 0:
            max_param = 0
        else:
            max_param = max(np.concatenate(current_params).max(),knot_param) 
        knot_param += np.linalg.norm(endpoints[1] - endpoints[0])
        if len(param[argsort]) > 0:
            current_params.append(param[argsort] + max_param)
            current_points.append(curve_data[ind][truncate][argsort])
            current_distances.append(curve_distances[ind][truncate][argsort])
    if len(path) < 3 and len(curve_data[ind]) < 5:
            continue

#     print(len(current_points))
    try:
        
        path_points.append(np.concatenate(current_points))
        path_distances.append(np.concatenate(current_distances))
        path_params.append(np.concatenate(current_params))
        path_knots.append(np.linspace(0,np.concatenate(current_params).max(), max(5, int(len(current_points)/2.))))
        new_paths.append(path)
    except:
        continue
#     return path_points, path_distances, path_params, path_knots

In [43]:
paths = new_paths

In [44]:
path_angles = []
for path in paths:
    path_pairs = list(zip(path, path[1:]))
    current_angles = []
    if len(path_pairs) == 1:
        path_angles.append(None)
        continue
    for path_node in range(1,len(path) - 1):
        vec1 = np.array(corner_positions)[path[path_node]] - np.array(corner_positions)[path[0]]
        vec2 = np.array(corner_positions)[path[-1]] - np.array(corner_positions)[path[path_node]]
        vec1 /= np.linalg.norm(vec1)
        vec2 /= np.linalg.norm(vec2)
        current_angles.append(np.dot(vec1, vec2))
    path_angles.append(current_angles)
    
straight = np.zeros((len(paths)))
for i in range(len(paths)):
    if len(paths[i]) == 2:
        straight[i] = 1
    if path_angles[i] is not None:
        if (np.array(path_angles[i]) >= straight_threshold).all():
            straight[i] = 1

In [45]:
def spacing_weights(linspace):
    dists = linspace[1:]-linspace[:-1]
    return np.array( [dists[0]*0.5] + 
                       [(dists[i]+dists[i+1])*0.5 for i in range(0,len(dists)-1)]
                    +[dists[-1]*0.5])

In [46]:
def err(c, points, distances, u, t, k, endpoints):
    c = c.reshape(3,-1)
    eval_spline = np.array(splev(u, (t,c,k))).T
    proj_distances = np.linalg.norm(points - eval_spline, axis=1)
    residual = np.mean(np.abs(proj_distances - distances)) + 0.1*proj_distances.mean()
    return residual

In [47]:
inds = []
print('start splining')        
tcks = []
errs = []
straight_lines = []
closed_tcks = []
for i in tqdm(range(len(paths))):
    if len(path_params[i]) < 2:
        continue
    linspace = (path_params[i] - path_knots[i].min()) / (path_knots[i].max() - path_knots[i].min())
    where = np.unique(linspace, return_index=True)[1]
    linspace = linspace[where]
    argsort = np.argsort(linspace)
    linspace = linspace[argsort]
    path_points_current = path_points[i][where][argsort]
    knots = np.sort(((path_knots[i] - path_knots[i].min()) / (path_knots[i].max() - path_knots[i].min())))
    knots_as_needed = np.zeros((6+len(knots)))
    knots_as_needed[-3:] = 1
    knots_as_needed[3:-3] = knots
    weights = 1 - path_distances[i][where][argsort]
    space_weights = spacing_weights(linspace)
    endpoints = np.array(corner_positions)[np.array(paths[i])[[0,-1]]]
    if straight[i] == 1:
        straight_lines.append(endpoints)
        continue
    inds.append(i)
    try:
        (t,c0,k), u = splprep(u=linspace, x=path_points_current.T, w=space_weights, task=-1, t=knots_as_needed)
    except:
        continue
    con = ({'type': 'eq',
           'fun': lambda c: (splev(0, (t, c.reshape(3,-1), k))-endpoints[0])
           },
           {'type': 'eq',
           'fun': lambda c: (splev(1, (t, c.reshape(3,-1), k))-endpoints[1])
           })
    opt = minimize(err, np.array(c0).flatten(), (path_points_current, (1-weights), u, t, k, endpoints),
                  constraints=con, tol=1e-10)
    copt = opt.x
    tcks.append((t, copt.reshape(3,-1), k))

start splining


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14/14 [00:04<00:00,  2.94it/s]


In [48]:
# new_paths = []
path_points = []
path_distances = []
path_params = []
path_knots = []
avg_len = np.linalg.norm(np.array(corner_positions)[
    np.array(corner_pairs)[:,0]] - np.array(corner_positions)[
    np.array(corner_pairs)[:,1]], axis=1).mean()
for path in closed_paths:
    path_len = len(path)
#     print(len(path))
    path_pairs = list(zip(path, path[1:]))
    current_points = []
    current_distances = []
    current_params = []
    current_knots = []
    knot_param = 0
    for i,current_pair in enumerate(path_pairs):
        endpoints = np.array(corner_positions)[np.array(current_pair)]
        ind = np.where(np.isin(corner_pairs, current_pair).sum(1) == 2)[0][0]
        if len(path) < 3 and len(curve_data[ind]) < 5:
            break
        cosines = np.dot(curve_data[ind] - endpoints[0], endpoints[1] - endpoints[0]) / np.linalg.norm(endpoints[1] - endpoints[0])
        truncate = np.logical_and(cosines >= 0, cosines <= np.linalg.norm(endpoints[1] - endpoints[0]))

        param = cosines[truncate]

        argsort = np.argsort(param)
        if len(current_params) == 0:
            max_param = 0
        else:
            max_param = max(np.concatenate(current_params).max(),knot_param) 
        knot_param += np.linalg.norm(endpoints[1] - endpoints[0])
        if len(param[argsort]) > 0:
            current_params.append(param[argsort] + max_param)
            current_points.append(curve_data[ind][truncate][argsort])
            current_distances.append(curve_distances[ind][truncate][argsort])
    if len(path) < 3 and len(curve_data[ind]) < 5:
            continue
#     new_paths.append(path)
    path_points.append(np.concatenate(current_points))
    path_distances.append(np.concatenate(current_distances))
    path_params.append(np.concatenate(current_params))
    path_knots.append(np.linspace(0,np.concatenate(current_params).max(),6))

In [49]:
if len(closed_paths) > 0:
    print('splining closed curves')
    
    for i in tqdm(range(len(closed_paths))):
        linspace = (path_params[i] - path_knots[i].min()) / (path_knots[i].max() - path_knots[i].min())
        where = np.unique(linspace, return_index=True)[1]
        linspace = linspace[where]
        argsort = np.argsort(linspace)
        linspace = linspace[argsort]
        path_points_current = path_points[i][where][argsort]
        knots = np.sort(((path_knots[i] - path_knots[i].min()) / (path_knots[i].max() - path_knots[i].min())))
        knots_as_needed = np.zeros((6+len(knots)))
        knots_as_needed[-3:] = 1
        knots_as_needed[3:-3] = knots
        weights = 1 - path_distances[i][where][argsort]
        space_weights = spacing_weights(linspace)

        (t,c0,k), u = splprep(u=linspace, x=path_points_current.T, w=space_weights, task=-1, t=knots_as_needed)
        con = ({'type': 'eq',
               'fun': lambda c: (np.array(splev(0, (t, c.reshape(3,-1), k)))-np.array(splev(1, (t, c.reshape(3,-1), k))))
               },
               {'type': 'eq',
               'fun': lambda c: (np.sum(np.abs(np.array(splev(0, (t, c.reshape(3,-1), k), der=1))-np.array(splev(1, (t, c.reshape(3,-1), k), der=1)))))
               }
              )
        opt = minimize(err, np.array(c0).flatten(), (path_points_current, 1-weights, u, t, k, endpoints),
                      constraints=con)
        copt = opt.x
        closed_tcks.append((t, copt.reshape(3,-1), k))


In [50]:
# closed_tcks

In [51]:
plot = k3d.plot()

cmap=k3d.colormaps.matplotlib_color_maps.plasma_r
colors = k3d.helpers.map_colors(
                distances, cmap, [distances.min(), distances.max()]
            ).astype(np.uint32)

k3d_points = k3d.points(points, point_size=DISPLAY_RES, opacity=0.1, shader='3d', name='sharp_points')
plot += k3d_points

for i in range(len(straight_lines)):
    spline = k3d.line(straight_lines[i], color=0xff0000, width=DISPLAY_RES-0.015, name='line')
    plot += spline

for i in range(len(tcks)):
    spline = k3d.points(np.array(splev(np.linspace(0,1,2500), tcks[i])).T, color=0xff0000, point_size=DISPLAY_RES-0.015, shader='flat', name='default')
    plot += spline
    
for i in range(len(closed_tcks)):    
    spline = k3d.points(np.array(splev(np.linspace(0,1,2500), closed_tcks[i])).T, color=0xff0000, point_size=DISPLAY_RES-0.015, shader='flat', name='spline')
    plot += spline

plot.display()

with open(meta_exp_name+'/'+folder_name+'/'+'vectorization_'+curve_extraction_mode+'.html', 'w') as f:
    f.write(plot.get_snapshot())

Output()

In [52]:
import pickle
def write_results(folder_name,straight_lines,tcks,closed_tcks):
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
    with open(folder_name + '/vectorization_line.txt', "wb") as fp:   #Pickling
        pickle.dump(straight_lines, fp)
    with open(folder_name+'/vectorization_open_curves.txt', "wb") as fp:   #Pickling
        pickle.dump(tcks, fp)
    with open(folder_name+'/vectorization_closed_curves.txt', "wb") as fp:   #Pickling
        pickle.dump(closed_tcks, fp)
        
def read_results(folder_name):
    with open(folder_name + '/vectorization_line.txt', "rb") as fp:   # Unpickling
        straight_lines = pickle.load(fp)
    with open(folder_name+'/vectorization_open_curves.txt', "rb") as fp:   #Pickling
        tcks = pickle.load(fp)
    with open(folder_name+'/vectorization_closed_curves.txt', "rb") as fp:   #Pickling
        closed_tcks = pickle.load(fp)
    return straight_lines,tcks,closed_tcks

In [53]:
write_results(meta_exp_name+'/'+folder_name+'/raw_vectorization/',straight_lines,tcks,closed_tcks)

# Metric calculation

In [54]:
def loss_calculating(pred_sampling_small,inp_sampling_small,blur=0.05):
    loss = SamplesLoss(loss="sinkhorn",backend = "tensorized",blur=blur)
    sink_ls = loss(pred_sampling_small.cuda(), inp_sampling_small.cuda())

    loss = SamplesLoss(loss="energy",backend = "tensorized",blur=blur)
    en_ls = loss(pred_sampling_small.cuda(), inp_sampling_small.cuda())

    loss = SamplesLoss(loss="gaussian",backend = "tensorized",blur=blur)
    gaus_ls = loss(pred_sampling_small.cuda(), inp_sampling_small.cuda())

    loss = SamplesLoss(loss="laplacian",backend = "tensorized",blur=blur)
    lap_ls = loss(pred_sampling_small.cuda(), inp_sampling_small.cuda())

    return sink_ls, en_ls, gaus_ls, lap_ls

In [55]:
def write_to_file(folder_name, file_name, CD_gt_to_pred, CD_pred_to_gt,chamf_l, sink_ls, en_ls, gaus_ls, lap_ls):
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
    with open(folder_name +'/'+'metrics_'+ file_name + curve_extraction_mode + '.txt', 'w') as f:
        f.write('CD_gt_to_pred')
        f.write('\n')
        f.write(str(CD_gt_to_pred.item()))
        f.write('\n')
        f.write('CD_pred_to_gt')
        f.write('\n')
        f.write(str(CD_pred_to_gt.item()))
        f.write('\n')
        f.write('Chamfer loss')
        f.write('\n')
        f.write(str(chamf_l))
        f.write('\n')
        f.write('Energy loss')
        f.write('\n')
        f.write(str(en_ls.item()))
        f.write('\n')
        f.write('Sinkhorn loss')
        f.write('\n')
        f.write(str(sink_ls.item()))
        f.write('\n')
        f.write('Gaussian loss')
        f.write('\n')
        f.write(str(gaus_ls.item()))
        f.write('\n')
        f.write('Laplassian loss')
        f.write('\n')
        f.write(str(lap_ls.item()))
        f.write('\n')

In [56]:
import os
NO_GT=False
# load GT curves as segments
gt_edges = []
if os.path.exists('parametric_labeling/{file}__parametric.txt'.format(file=_id[9:])):
    with open('parametric_labeling/{file}__parametric.txt'.format(file=_id[9:])) as f:
        for line in f:
            curve_id, curve_type, edges = line.split(' ', maxsplit=2)
            gt_edges.append(np.array([float(v) for v in edges.split()]).reshape((-1, 2, 3)))
else:
    NO_GT = True
    print('No GT!')

In [57]:
# sample GT edges
if not NO_GT:
    gt_sampling = []
    for edge in np.concatenate(gt_edges):
        num = int(np.linalg.norm(edge[1] - edge[0]) // 0.001)
        if num > 0:
            linspace = np.linspace(0, 1, num)
        #     break
            gt_sampling.append(linspace[:,None] * edge[0] + (1 - linspace)[:,None] * edge[1])
        else:
            gt_sampling.append(edge)
    gt_sampling = np.concatenate(gt_sampling)

In [58]:
def pred_sampling_points(straight_lines,tcks,closed_tcks):
    pred_sampling = []

    for i in range(len(straight_lines)):
        edge = straight_lines[i]
        num = int(np.linalg.norm(edge[1] - edge[0]) // 0.001)
        if num > 0:
            linspace = np.linspace(0, 1, num)
        #     break
            pred_sampling.append(linspace[:,None] * edge[0] + (1 - linspace)[:,None] * edge[1])
        else:
            pred_sampling.append(edge)

    for i in range(len(tcks)):
        pred_sampling.append(np.array(splev(np.linspace(0,1,5000), tcks[i])).T)

    for i in range(len(closed_tcks)):    
        pred_sampling.append(np.array(splev(np.linspace(0,1,5000), closed_tcks[i])).T)
    return pred_sampling

In [59]:
def metric_calculation(pred_sampling,gt_sampling,blur=0.05):
  
    CD_gt_to_pred = (cKDTree(pred_sampling).query(gt_sampling, 1)[0]**2).mean()
    CD_pred_to_gt = (cKDTree(gt_sampling).query(pred_sampling, 1)[0]**2).mean()
    
    pred_sampling = torch.tensor(pred_sampling)[None]
    gt_sampling = torch.tensor(gt_sampling)[None]
    chamf_l = chamfer_distance(pred_sampling.clone().cuda().detach().float(), gt_sampling.clone().cuda().detach().float())
    
    if pred_sampling[0].shape[0]>10000:
        sub_idx = pcu.downsample_point_cloud_poisson_disk(np.array(pred_sampling[0]), num_samples=10000)
        # Use the indices to get the sample positions and normals
        pred_sampling_small = pred_sampling[:,sub_idx]
    else:
        pred_sampling_small =  pred_sampling
    
    if gt_sampling[0].shape[0]>10000:
        sub_idx = pcu.downsample_point_cloud_poisson_disk(np.array(gt_sampling[0]), num_samples=10000)
        # Use the indices to get the sample positions and normals
        gt_sampling_small = gt_sampling[:,sub_idx]
    else:
        gt_sampling_small = gt_sampling
    sink_ls, en_ls, gaus_ls, lap_ls = loss_calculating(pred_sampling_small,gt_sampling_small,blur=blur)
    return CD_gt_to_pred, CD_pred_to_gt,chamf_l, sink_ls, en_ls, gaus_ls, lap_ls

In [60]:
pred_sampling = pred_sampling_points(straight_lines,tcks,closed_tcks)
pred_sampling = np.concatenate(pred_sampling)


In [61]:
if not NO_GT:
    CD_gt_to_pred, CD_pred_to_gt,chamf_l, sink_ls, en_ls, gaus_ls, lap_ls  = metric_calculation(pred_sampling,gt_sampling)
    _, _,chamf_l, sink_ls_bl02, en_ls_bl02, gaus_ls_bl02, lap_ls_bl02  = metric_calculation(pred_sampling,gt_sampling)

In [62]:
if not NO_GT:
    write_to_file(meta_exp_name+'/'+folder_name+'/metrics/','raw_vectorization',CD_gt_to_pred, CD_pred_to_gt,chamf_l, sink_ls, en_ls, gaus_ls, lap_ls)
    df = pd.DataFrame([[folder_name,CD_gt_to_pred.item(),CD_pred_to_gt.item(),chamf_l[0].item(),
                        en_ls.item(),sink_ls.item(),
                        gaus_ls.item(),lap_ls.item(), en_ls_bl02.item(),sink_ls_bl02.item(),
                        gaus_ls_bl02.item(),lap_ls_bl02.item()]],
                      index=None,  columns=['Name','CD_gt_to_pred','CD_pred_to_gt','Chamfer_loss',
                                            'Energy_loss','Sinkhorn_loss','Gaussian_loss'
                                            ,'Laplassian_loss','Energy_loss_blure02','Sinkhorn_loss_blure02','Gaussian_loss_blure02'
                                            ,'Laplassian_loss_blure02'])
    if not os.path.exists(meta_exp_name+'/metrics/'):
        os.makedirs(meta_exp_name+'/metrics/')
    
    if not os.path.exists(meta_exp_name+'/metrics/'+meta_exp_name + '_raw_vectorization_metrics.csv'):
        df.to_csv(meta_exp_name+'/metrics/'+meta_exp_name + '_raw_vectorization_metrics.csv', index = False,sep=';')
    else:
        df.to_csv(meta_exp_name+'/metrics/'+meta_exp_name + '_raw_vectorization_metrics.csv', mode='a', header=None, index = False,sep=';')

In [63]:
# save this also
if not NO_GT:
    plot = k3d.plot()

    cmap=k3d.colormaps.matplotlib_color_maps.plasma_r
    colors = k3d.helpers.map_colors(
                    distances, cmap, [distances.min(), distances.max()]
                ).astype(np.uint32)

    k3d_points = k3d.points(pred_sampling, point_size=DISPLAY_RES, opacity=0.1, shader='3d', name='sharp_points', color=0xff0000)
    plot += k3d_points

    k3d_points = k3d.points(gt_sampling, point_size=DISPLAY_RES, opacity=0.1, shader='3d', name='sharp_points')
    plot += k3d_points

    plot.display()
    with open(meta_exp_name+'/'+folder_name+'/'+'vectorization_with_GT_'+curve_extraction_mode+'.html', 'w') as f:
        f.write(plot.get_snapshot())

Output()

### Caclulating distance between prediction and input point cloud

In [64]:
CD_gt_to_pred_mm, CD_pred_to_gt_mm, chamf_l_mm, sink_ls_mm, en_ls_mm, gaus_ls_mm, lap_ls_mm  = metric_calculation(pred_sampling,points)

In [65]:
write_to_file(meta_exp_name+'/'+folder_name+'/metrics/','chosing_raw_vectorization',CD_gt_to_pred_mm, CD_pred_to_gt_mm, chamf_l_mm, sink_ls_mm, en_ls_mm, gaus_ls_mm, lap_ls_mm)

df = pd.DataFrame([[folder_name,chamf_l_mm[0].item(),en_ls_mm.item(),sink_ls_mm.item(),gaus_ls_mm.item(),lap_ls_mm.item()]],
                  index=None,  columns=['Name','Chamfer_loss','Energy_loss','Sinkhorn_loss','Gaussian_loss','Laplassian_loss'])

if not os.path.exists(meta_exp_name+'/metrics/'):
    os.makedirs(meta_exp_name+'/metrics/')
    
if not os.path.exists(meta_exp_name+'/metrics/' + meta_exp_name + '_chosing_raw_vectorization_metrics.csv'):
    df.to_csv(meta_exp_name+'/metrics/'+meta_exp_name + '_chosing_raw_vectorization_metrics.csv', index = False,sep=';')
else:
    df.to_csv(meta_exp_name+'/metrics/'+meta_exp_name + '_chosing_raw_vectorization_metrics.csv', mode='a', header=None, index = False,sep=';')

# Pretification

In [66]:
# straight_lines,tcks,closed_tcks = read_results(meta_exp_name+'/'+folder_name+'/raw_vectorization/')
# cut_metric ='fscore'

In [67]:
#0.00001

def fscorebeta(dist1, dist2, threshold=0.005, beta = 0.5):
    # From here https://github.com/ThibaultGROUEIX/ChamferDistancePytorch/blob/master/fscore.py
    """
    Calculates the F-score between two point clouds with the corresponding threshold value.
    
    :param dist1: Batch, N-Points
    :param dist2: Batch, N-Points
    :param th: float
    :return: fscore, precision, recall
    """
    # NB : In this depo, dist1 and dist2 are squared pointcloud euclidean distances, so you should adapt the threshold accordingly.
    precision_1 = torch.mean((dist1 < threshold).float(), dim=1)
    precision_2 = torch.mean((dist2 < threshold).float(), dim=1)
    fscore = (1 + beta*beta) * precision_1 * precision_2 / (beta*beta*precision_1 + precision_2)
    fscore[torch.isnan(fscore)] = 0
    return fscore, precision_1, precision_2

In [68]:
pred_sampling = pred_sampling_points(straight_lines,tcks,closed_tcks)
pred_sampling = np.concatenate(pred_sampling)

In [69]:
if cut_metric =='fscore' or cut_metric =='precision':
    CD_gt_to_pred = (cKDTree(pred_sampling).query(points, 1)[0]**2)
    CD_pred_to_gt = (cKDTree(points).query(pred_sampling, 1)[0]**2)
else:
    pred_sampling = torch.tensor(pred_sampling)[None]

In [70]:
if cut_metric =='fscore'  or cut_metric =='precision':
    fsc_max,pred_1_max,pred_2_max = fscorebeta(torch.tensor(CD_pred_to_gt)[None],torch.tensor(CD_gt_to_pred)[None]) 
elif cut_metric == 'chamfer_dist':
    ch_dst_min = chamf_l_mm[0].item()
elif cut_metric == 'sinkhorn':
    sink_dst_min = sink_ls_mm.item()

In [71]:
it=0
curve_cuted =0
while it<len(straight_lines)+len(tcks)+len(closed_tcks):
    pred_sampling = []
    tks =  copy.deepcopy(tcks)
    str_lines = copy.deepcopy(straight_lines)
    closed_tks = copy.deepcopy(closed_tcks)
    if it < len(straight_lines):
        str_lines.pop(it)
    elif it < len(straight_lines)+len(tcks):
        tks.pop(it-len(straight_lines))   
    else:
        closed_tks.pop(it-len(straight_lines)-len(tcks))
    
    if len(str_lines)+len(tks)+len(closed_tks)==0:
        straight_lines = copy.deepcopy(str_lines)
        tcks = copy.deepcopy(tks)
        closed_tcks = copy.deepcopy(closed_tks)
        curve_cuted+=1
        break
        
    pred_sampling = pred_sampling_points(str_lines,tks,closed_tks)
    pred_sampling = np.concatenate(pred_sampling)
    
    if cut_metric =='fscore' or cut_metric =='precision':
        CD_gt_to_pred = (cKDTree(pred_sampling).query(points, 1)[0]**2)
        CD_pred_to_gt = (cKDTree(points).query(pred_sampling, 1)[0]**2)
        fsc,pred_1,pred_2  = fscorebeta(torch.tensor(CD_pred_to_gt)[None],torch.tensor(CD_gt_to_pred)[None]) 
        print(fsc,pred_1,pred_2)
        if cut_metric =='precision':
            if pred_1 > pred_1_max:
                fsc_max = fsc
                pred_1_max = pred_1
                pred_2_max = pred_2
                straight_lines = copy.deepcopy(str_lines)
                tcks = copy.deepcopy(tks)
                closed_tcks = copy.deepcopy(closed_tks)
                curve_cuted+=1
            else:
                it+=1
        else:
            if fsc > fsc_max:
                fsc_max = fsc
                pred_2_max = pred_2
                pred_1_max = pred_1
                straight_lines = copy.deepcopy(str_lines)
                tcks = copy.deepcopy(tks)
                closed_tcks = copy.deepcopy(closed_tks)
                curve_cuted+=1
            else:
                it+=1
    elif cut_metric == 'chamfer_dist':
        pred_sampling = torch.tensor(pred_sampling)[None]
        chamf_l_mm = chamfer_distance(pred_sampling.clone().cuda().detach().float(), torch.tensor(points)[None].clone().cuda().detach().float())
        if chamf_l_mm[0].item()<=ch_dst_min:
            ch_dst_min = chamf_l_mm[0].item()
            straight_lines = copy.deepcopy(str_lines)
            tcks = copy.deepcopy(tks)
            closed_tcks = copy.deepcopy(closed_tks)
            curve_cuted+=1
        else:
            it+=1
    elif cut_metric == 'sinkhorn':
        pred_sampling = torch.tensor(pred_sampling)[None]
        if pred_sampling[0].shape[0]>10000:
            sub_idx = pcu.downsample_point_cloud_poisson_disk(np.array(pred_sampling[0]), num_samples=10000)
            pred_sampling_small = pred_sampling[:,sub_idx]
        else:
            pred_sampling_small =  pred_sampling
        loss = SamplesLoss(loss="sinkhorn",backend = "tensorized")
        sink_ls_mm = loss(pred_sampling_small.cuda(), inp_sampling_small.cuda())
        if  sink_ls_mm.item()<=sink_dst_min:
            sink_dst_min= sink_ls_mm.item()
            straight_lines = copy.deepcopy(str_lines)
            tcks = copy.deepcopy(tks)
            closed_tcks = copy.deepcopy(closed_tks)
            curve_cuted+=1
        else:
            it+=1
print('Get rid of:', curve_cuted)

tensor([0.9802]) tensor([0.9936]) tensor([0.9299])
tensor([0.9753]) tensor([0.9936]) tensor([0.9084])
tensor([0.9860]) tensor([0.9937]) tensor([0.9561])
tensor([0.9788]) tensor([0.9936]) tensor([0.9236])
tensor([0.9817]) tensor([0.9936]) tensor([0.9369])
tensor([0.9751]) tensor([0.9936]) tensor([0.9077])
tensor([0.9790]) tensor([0.9936]) tensor([0.9245])
tensor([0.9643]) tensor([1.]) tensor([0.8438])
tensor([0.9712]) tensor([0.9929]) tensor([0.8931])
tensor([0.9577]) tensor([0.9929]) tensor([0.8387])
tensor([0.9762]) tensor([0.9929]) tensor([0.9143])
tensor([0.9723]) tensor([0.9929]) tensor([0.8976])
tensor([0.9755]) tensor([0.9929]) tensor([0.9112])
tensor([0.9754]) tensor([0.9929]) tensor([0.9110])
Get rid of: 0


In [72]:
plot = k3d.plot()

cmap=k3d.colormaps.matplotlib_color_maps.plasma_r
colors = k3d.helpers.map_colors(
                distances, cmap, [distances.min(), distances.max()]
            ).astype(np.uint32)

k3d_points = k3d.points(points, point_size=DISPLAY_RES, opacity=0.1, shader='3d', name='sharp_points')
plot += k3d_points

for i in range(len(straight_lines)):
    spline = k3d.line(straight_lines[i], color=0xff0000, width=DISPLAY_RES-0.015, name='line')
    plot += spline

for i in range(len(tcks)):
    spline = k3d.points(np.array(splev(np.linspace(0,1,2500), tcks[i])).T, color=0xff0000, point_size=DISPLAY_RES-0.015, shader='flat', name='default')
    plot += spline
    
for i in range(len(closed_tcks)):    
    spline = k3d.points(np.array(splev(np.linspace(0,1,2500), closed_tcks[i])).T, color=0xff0000, point_size=DISPLAY_RES-0.015, shader='flat', name='spline')
    plot += spline

plot.display()

with open(meta_exp_name+'/'+folder_name+'/'+'pretified_vectorization_'+curve_extraction_mode+'.html', 'w') as f:
    f.write(plot.get_snapshot())

Output()

# Writing metrics and curves

In [73]:
# sample GT edges
if not NO_GT:
    gt_sampling = []
    for edge in np.concatenate(gt_edges):
        num = int(np.linalg.norm(edge[1] - edge[0]) // 0.001)
        if num > 0:
            linspace = np.linspace(0, 1, num)
        #     break
            gt_sampling.append(linspace[:,None] * edge[0] + (1 - linspace)[:,None] * edge[1])
        else:
            gt_sampling.append(edge)
    gt_sampling = np.concatenate(gt_sampling)

In [74]:
write_results(meta_exp_name+'/'+folder_name+'/pretified_vectorization/',straight_lines,tcks,closed_tcks)

In [75]:
pred_sampling = pred_sampling_points(straight_lines,tcks,closed_tcks)
pred_sampling = np.concatenate(pred_sampling)
if not NO_GT:
    CD_gt_to_pred, CD_pred_to_gt,chamf_l, sink_ls, en_ls, gaus_ls, lap_ls  = metric_calculation(pred_sampling,gt_sampling)
    _, _,chamf_l, sink_ls_bl02, en_ls_bl02, gaus_ls_bl02, lap_ls_bl02  = metric_calculation(pred_sampling,gt_sampling)

In [76]:
if not NO_GT:
    write_to_file(meta_exp_name+'/'+folder_name+'/metrics/','pretified_vectorization',CD_gt_to_pred, CD_pred_to_gt,chamf_l, sink_ls, en_ls, gaus_ls, lap_ls)
    df = pd.DataFrame([[folder_name,CD_gt_to_pred.item(),CD_pred_to_gt.item(),chamf_l[0].item(),
                        en_ls.item(),sink_ls.item(),
                        gaus_ls.item(),lap_ls.item(), en_ls_bl02.item(),sink_ls_bl02.item(),
                        gaus_ls_bl02.item(),lap_ls_bl02.item()]],
                      index=None,  columns=['Name','CD_gt_to_pred','CD_pred_to_gt','Chamfer_loss',
                                            'Energy_loss','Sinkhorn_loss','Gaussian_loss'
                                            ,'Laplassian_loss','Energy_loss_blure02','Sinkhorn_loss_blure02','Gaussian_loss_blure02'
                                            ,'Laplassian_loss_blure02'])
    if not os.path.exists(meta_exp_name+'/metrics/'):
        os.makedirs(meta_exp_name+'/metrics/')
    
    if not os.path.exists(meta_exp_name+'/metrics/'+meta_exp_name + '_pretified_vectorization_metrics.csv'):
        df.to_csv(meta_exp_name+'/metrics/'+meta_exp_name + '_pretified_vectorization_metrics.csv', index = False,sep=';')
    else:
        df.to_csv(meta_exp_name+'/metrics/'+meta_exp_name + '_pretified_vectorization_metrics.csv', mode='a', header=None, index = False,sep=';')

In [77]:
CD_gt_to_pred_mm, CD_pred_to_gt_mm, chamf_l_mm, sink_ls_mm, en_ls_mm, gaus_ls_mm, lap_ls_mm  = metric_calculation(pred_sampling,points)

In [78]:
write_to_file(meta_exp_name+'/'+folder_name+'/metrics/','chosing_pretified_vectorization',CD_gt_to_pred_mm, CD_pred_to_gt_mm, chamf_l_mm, sink_ls_mm, en_ls_mm, gaus_ls_mm, lap_ls_mm)

df = pd.DataFrame([[folder_name,chamf_l_mm[0].item(),en_ls_mm.item(),sink_ls_mm.item(),gaus_ls_mm.item(),lap_ls_mm.item()]],
                  index=None,  columns=['Name','Chamfer_loss','Energy_loss','Sinkhorn_loss','Gaussian_loss','Laplassian_loss'])

if not os.path.exists(meta_exp_name+'/metrics/'):
    os.makedirs(meta_exp_name+'/metrics/')
    
if not os.path.exists(meta_exp_name+'/metrics/' + meta_exp_name + '_chosing_pretified_vectorization_metrics.csv'):
    df.to_csv(meta_exp_name+'/metrics/'+meta_exp_name + '_chosing_pretified_vectorization_metrics.csv', index = False,sep=';')
else:
    df.to_csv(meta_exp_name+'/metrics/'+meta_exp_name + '_chosing_pretified_vectorization_metrics.csv', mode='a', header=None, index = False,sep=';')

# Small lines cutting

In [79]:
def curve_length(curve,devision_points_count=200):
    """
    input:
        curve - curve in as in scipy splprep
        devision_points_count - the amount of points on curve
    output:
        length of the curve
    """
    ar = np.array(splev(np.linspace(0,1,devision_points_count), curve)).T
    return np.sum(np.sqrt(np.sum((ar[:-1]-ar[1:])**2,axis=1)))

### Adding curves endpoints as lines for constructing connection graph

In [80]:
str_lines = copy.deepcopy(straight_lines) 
for it in tcks:
    ar = np.array(splev(np.linspace(0,1,2), it)).T
    str_lines.append(ar)

### Constructing graph where vertex are primitives and edges between vertex exist if primitives share endpoints

In [81]:
import networkx as nx
G = nx.Graph()
G.add_nodes_from(range(len(str_lines)))

for i,line in enumerate(str_lines):
        for j,line_2 in  enumerate(str_lines):
            if j>i:
                if np.allclose(line[0],line_2[0]) or np.allclose(line[0],line_2[1]) or np.allclose(line[1],line_2[0]) or np.allclose(line[1],line_2[1]):
                    G.add_edge(i,j)

### Getting connected subgraphs 

In [82]:
S = [G.subgraph(c).copy() for c in nx.connected_components(G)]

### Calculating for each length of  conected segments

In [83]:
gr = []
for sg in S:
    gr_len=0
    for node in sg.nodes():
        if node<len(straight_lines):
            #for lines
            gr_len+=np.sqrt(np.sum((straight_lines[node][0]-straight_lines[node][1])**2))
        else:
            #for curves
            gr_len+=curve_length(tcks[node-len(straight_lines)])
    gr.append(gr_len)

## Getting read of small curves and lines which are in small component

In [84]:
str_line_copy = []
tcks_copy = []
for i,sg in enumerate(S):
    if gr[i]<small_lines_clip_primitives_length and len(sg.nodes)<=small_lines_clip_primitive_number:
        continue
    for node in sg.nodes():
        if node<len(straight_lines):
            #for lines
            str_line_copy.append(straight_lines[node])
        else:
            #for curves
            tcks_copy.append(tcks[node-len(straight_lines)])

## Constructing result

In [85]:
straight_lines = str_line_copy
tcks = tcks_copy

In [86]:
plot = k3d.plot()

cmap=k3d.colormaps.matplotlib_color_maps.plasma_r
colors = k3d.helpers.map_colors(
                distances, cmap, [distances.min(), distances.max()]
            ).astype(np.uint32)

k3d_points = k3d.points(points, point_size=DISPLAY_RES, opacity=0.1, shader='3d', name='sharp_points')
plot += k3d_points

for i in range(len(straight_lines)):
    spline = k3d.line(str_line_copy[i], color=0xff0000, width=DISPLAY_RES-0.015, name='line')
    plot += spline

for i in range(len(tcks)):
    spline = k3d.points(np.array(splev(np.linspace(0,1,2500), tcks[i])).T, color=0xff0000, point_size=DISPLAY_RES-0.015, shader='flat', name='default')
    plot += spline
    
for i in range(len(closed_tcks)):    
    spline = k3d.points(np.array(splev(np.linspace(0,1,2500), closed_tcks[i])).T, color=0xff0000, point_size=DISPLAY_RES-0.015, shader='flat', name='spline')
    plot += spline

plot.display()

with open(meta_exp_name+'/'+folder_name+'/'+'pretified_vectorization_final_'+curve_extraction_mode+'.html', 'w') as f:
    f.write(plot.get_snapshot())

Output()

# Writing metrics and curves

In [84]:
# sample GT edges
if not NO_GT:
    gt_sampling = []
    for edge in np.concatenate(gt_edges):
        num = int(np.linalg.norm(edge[1] - edge[0]) // 0.001)
        if num > 0:
            linspace = np.linspace(0, 1, num)
        #     break
            gt_sampling.append(linspace[:,None] * edge[0] + (1 - linspace)[:,None] * edge[1])
        else:
            gt_sampling.append(edge)
    gt_sampling = np.concatenate(gt_sampling)

In [85]:
write_results(meta_exp_name+'/'+folder_name+'/pretified_vectorization_final/',str_line_copy,tcks,closed_tcks)

In [86]:
pred_sampling = pred_sampling_points(straight_lines,tcks,closed_tcks)
pred_sampling = np.concatenate(pred_sampling)
if not NO_GT:
    CD_gt_to_pred, CD_pred_to_gt,chamf_l, sink_ls, en_ls, gaus_ls, lap_ls  = metric_calculation(pred_sampling,gt_sampling)
    _, _,chamf_l, sink_ls_bl02, en_ls_bl02, gaus_ls_bl02, lap_ls_bl02  = metric_calculation(pred_sampling,gt_sampling)

In [87]:
if not NO_GT:
    write_to_file(meta_exp_name+'/'+folder_name+'/metrics/','pretified_vectorization_final',CD_gt_to_pred, CD_pred_to_gt,chamf_l, sink_ls, en_ls, gaus_ls, lap_ls)
    df = pd.DataFrame([[folder_name,CD_gt_to_pred.item(),CD_pred_to_gt.item(),chamf_l[0].item(),
                        en_ls.item(),sink_ls.item(),
                        gaus_ls.item(),lap_ls.item(), en_ls_bl02.item(),sink_ls_bl02.item(),
                        gaus_ls_bl02.item(),lap_ls_bl02.item()]],
                      index=None,  columns=['Name','CD_gt_to_pred','CD_pred_to_gt','Chamfer_loss',
                                            'Energy_loss','Sinkhorn_loss','Gaussian_loss'
                                            ,'Laplassian_loss','Energy_loss_blure02','Sinkhorn_loss_blure02','Gaussian_loss_blure02'
                                            ,'Laplassian_loss_blure02'])
    if not os.path.exists(meta_exp_name+'/metrics/'):
        os.makedirs(meta_exp_name+'/metrics/')
    
    if not os.path.exists(meta_exp_name+'/metrics/'+meta_exp_name + '_pretified_vectorization_final_metrics.csv'):
        df.to_csv(meta_exp_name+'/metrics/'+meta_exp_name + '_pretified_vectorization_final_metrics.csv', index = False,sep=';')
    else:
        df.to_csv(meta_exp_name+'/metrics/'+meta_exp_name + '_pretified_vectorization_final_metrics.csv', mode='a', header=None, index = False,sep=';')

In [88]:
CD_gt_to_pred_mm, CD_pred_to_gt_mm, chamf_l_mm, sink_ls_mm, en_ls_mm, gaus_ls_mm, lap_ls_mm  = metric_calculation(pred_sampling,points)

In [89]:
write_to_file(meta_exp_name+'/'+folder_name+'/metrics/','chosing_pretified_vectorization_final',CD_gt_to_pred_mm, CD_pred_to_gt_mm, chamf_l_mm, sink_ls_mm, en_ls_mm, gaus_ls_mm, lap_ls_mm)

df = pd.DataFrame([[folder_name,chamf_l_mm[0].item(),en_ls_mm.item(),sink_ls_mm.item(),gaus_ls_mm.item(),lap_ls_mm.item()]],
                  index=None,  columns=['Name','Chamfer_loss','Energy_loss','Sinkhorn_loss','Gaussian_loss','Laplassian_loss'])

if not os.path.exists(meta_exp_name+'/metrics/'):
    os.makedirs(meta_exp_name+'/metrics/')
    
if not os.path.exists(meta_exp_name+'/metrics/' + meta_exp_name + '_chosing_pretified_vectorization_final_metrics.csv'):
    df.to_csv(meta_exp_name+'/metrics/'+meta_exp_name + '_chosing_pretified_vectorization_final_metrics.csv', index = False,sep=';')
else:
    df.to_csv(meta_exp_name+'/metrics/'+meta_exp_name + '_chosing_pretified_vectorization_final_metrics.csv', mode='a', header=None, index = False,sep=';')